<img src="images/Logo.png" style="height:75px;width:120px;" align="left">

<br><br><h1>Example of using TensorFlow to build a Classifier for Iris Data</h1>
<img src="images/Iris-Example-1.png">
<img src="images/Iris-Example-2.png">

<img src="images/Logo.png" style="height:75px;width:120px;" align="left">
<h2>Example of a Deep Neural Network (DNN) to classify Iris Petals into three Classes [0,1,2] </h2>
<h3> The code below is adapted from the TensorFlow example, with modifications </h3>


In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import shutil
from six.moves.urllib.request import urlopen

import numpy as np
import tensorflow as tf

# Data sets
IRIS_TRAINING = "iris_training.csv"
IRIS_TRAINING_URL = "http://download.tensorflow.org/data/iris_training.csv"

IRIS_TEST = "iris_test.csv"
IRIS_TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"

def iris_classifier_dnn(hidden_units=None, training_steps=2000, 
                    model_dir="/tmp/iris_model",
                    delete_old_model=False
                   ):
    
  # Set the model type to be SVM-{kernel}
  modelType = "DNN"
    
  # 0 represents setosa 1 represents versicolor 2 represents virginica
  iris_classes = ["setosa", "versicolor", "virginica"]
    
  # If the training and test sets aren't stored locally, download them.
  if not os.path.exists(IRIS_TRAINING):
    raw = urlopen(IRIS_TRAINING_URL).read()
    with open(IRIS_TRAINING, "wb") as f:
      f.write(raw)

  if not os.path.exists(IRIS_TEST):
    raw = urlopen(IRIS_TEST_URL).read()
    with open(IRIS_TEST, "wb") as f:
      f.write(raw)

  if delete_old_model:
    if os.path.exists(model_dir):
        shutil.rmtree(model_dir)
        print("Deleting old model {0}\n" .format(model_dir))
  # Load datasets.
  training_set = tf.contrib.learn.datasets.base.load_csv_with_header(
      filename=IRIS_TRAINING,
      target_dtype=np.int,
      features_dtype=np.float32)
  test_set = tf.contrib.learn.datasets.base.load_csv_with_header(
      filename=IRIS_TEST,
      target_dtype=np.int,
      features_dtype=np.float32)

  # Specify that all features have real-value data
  feature_columns = [tf.feature_column.numeric_column("x", 
                                                      shape=[training_set.data.shape[1]])]

  print("Featured Columns {0}\n" .format(feature_columns))                                                    
  # Build 3 layer DNN with 10, 20, 10 units respectively.
  print("Building DNN with {0} hidden units\n" .format(hidden_units))
  classifier = tf.estimator.DNNClassifier(feature_columns=feature_columns,
                                          hidden_units=hidden_units,
                                          n_classes=3,
                                          model_dir="/tmp/iris_model")
  # Define the training inputs
  train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": np.array(training_set.data)},
      y=np.array(training_set.target),
      num_epochs=None,
      shuffle=True)

  # Train model.
  classifier.train(input_fn=train_input_fn, steps=training_steps)

  # Define the test inputs
  test_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": np.array(test_set.data)},
      y=np.array(test_set.target),
      num_epochs=1,
      shuffle=False)

  # Evaluate accuracy.
  accuracy_score = classifier.evaluate(input_fn=test_input_fn)["accuracy"]

  print("\nTest Accuracy: {0:f}\n".format(accuracy_score))

  # Classify two new flower samples.
  new_samples = np.array(
      [[6.4, 3.2, 4.5, 1.5],
       [5.8, 3.1, 5.0, 1.7],
       [6.4 ,2.8, 5.6 ,2.2],
       [5.0 , 2.3 ,3.3 ,1.0 ],
       [4.9 ,2.5 ,4.5 ,1.7 ],
       [4.9 ,3.1 ,1.5 ,0.1 ],
       [5.7,3.8 ,1.7 ,0.3]], dtype=np.float32)
    
     
  predict_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": new_samples},
      num_epochs=1,
      shuffle=False)

  predictions = list(classifier.predict(input_fn=predict_input_fn))
  
  predicted_classes = [p["class_ids"] for p in predictions]
  # Store probabilities for each prediction  
  probs = [p["probabilities"] for p in predictions]
  probabilities = []
  predicted_class_names = []
  for i in range(len(predicted_classes)):
    predicted_classes[i] = int(predicted_classes[i])
    predicted_class_names.append(iris_classes[int(predicted_classes[i])])
    pr = probs[i]
    probabilities.append(pr[int(predicted_classes[i])]*100.0)

  print(
      "New Samples, Class Predictions:    {}\n"
      .format(predicted_classes))
  result= {"training-size" :  training_set.data.shape,
           "test-size" : test_set.data.shape,
           "train-steps" : training_steps,
           "model_dir"  : model_dir,
           "model-type" : modelType,
           "hidden-units" : str(hidden_units),
           "accuracy" : accuracy_score, 
           "prediction": predicted_classes,
           "prediction-names": predicted_class_names,
           "probabilities" : probabilities}

  return result  

<img src="images/Logo.png" style="height:75px;width:120px;" align="left">
<h2> Run the Code above and Below: This will create a DNN and run predictions on the Iris model data</h2>
    <ul><li> You can play with the hidden_units and training steps to see if this makes a difference to accuracy and predictions
    </li>
    </ul>

In [ ]:
import tensorflow as tf
print("TensorFlow Version {0}\n" .format(tf.__version__))
hidden_units = [10 , 20 , 10]
training_steps = 2000
model_dir = "/tmp/iris_model"
delete_old_model= True
result = iris_classifier_dnn(hidden_units=hidden_units, 
                         training_steps=training_steps, 
                         model_dir=model_dir,
                        delete_old_model=delete_old_model)
print("Result: {0}\n" .format(result))
print("Expected Results: [1, 2, 2, 1, 2, 0, 0]\n")
print("Expected Results: ['versicolor', 'virginica', 'virginica', 'versicolor', 'virginica', 'setosa', 'setosa']\n")


<img src="images/Logo.png" style="height:75px;width:120px;" align="left">
<ul>
<li>You should get an accuracy of 96.6 in the above DNN for Iris </li>
<li> You should get the following predictions:  [1, 2, 2, 1, 2, 0, 0]</li>
<li>You should get the follwing predictions: ['versicolor', 'virginica', 'virginica', 'versicolor', 'virginica', 'setosa', 'setosa']</li>
    </ul>


<img src="images/Logo.png" style="height:75px;width:120px;" align="left">

<h1> Example Of SVM for the Same Iris Classification</h1>
<ul>
<li> Modify the code below, and add other SVM kernels, such as "poly", "sigmoid" , "linear" and "rbf"</li>
<li> Modify the function iris_classifier_svm(kernel="poly") to take in a parameter for kernel='all'
<li> The function should return a JSON compaitable object (dictionary) that contains the results generated by each classifier
<li> Print out results from 'all' SVM kernels for the iris example, and print their predictions for each kernel</li>
<li> Add a decisionTree classified to the mix, and print out its prediction with the above 4  SVM kernels</li>

</ul>

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import shutil
from six.moves.urllib.request import urlopen
#import matplotlib.pyplot as plt
from sklearn import svm

import numpy as np
import pandas as pd
#import tensorflow as tf

# Data sets
IRIS_TRAINING = "iris_training.csv"
IRIS_TRAINING_URL = "http://download.tensorflow.org/data/iris_training.csv"

IRIS_TEST = "iris_test.csv"
IRIS_TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth',
                    'PetalLength', 'PetalWidth', 'Species']
label_name = 'Species'

def iris_classifier_svm(kernel='rbf',
                    model_dir="/tmp/iris_model",
                    delete_old_model=False
                   ):
    
  # Set the model type to be SVM-{kernel}
  modelType = "SVM-" + kernel
  # 0 represents setosa 1 represents versicolor 2 represents virginica
  iris_classes = ["setosa", "versicolor", "virginica"]
    
  # If the training and test sets aren't stored locally, download them.
  if not os.path.exists(IRIS_TRAINING):
    raw = urlopen(IRIS_TRAINING_URL).read()
    with open(IRIS_TRAINING, "wb") as f:
      f.write(raw)

  if not os.path.exists(IRIS_TEST):
    raw = urlopen(IRIS_TEST_URL).read()
    with open(IRIS_TEST, "wb") as f:
      f.write(raw)

  if delete_old_model:
    if os.path.exists(model_dir):
        shutil.rmtree(model_dir)
        print("Deleting old model {0}\n" .format(model_dir))
  # Load datasets.
  # Parse the local CSV file.
  training_df = pd.read_csv(filepath_or_buffer=IRIS_TRAINING,
                                names=CSV_COLUMN_NAMES,  # list of column names
                                header=0  # ignore the first row of the CSV file.
                               )
    # train now holds a pandas DataFrame, which is data structure
    # analogous to a table.

    # 1. Assign the DataFrame's labels (the right-most column) to train_label.
    # 2. Delete (pop) the labels from the DataFrame.
    # 3. Assign the remainder of the DataFrame to train_features
  train_features, train_label = training_df, training_df.pop(label_name)

  
  # Apply the preceding logic to the test set.
  
  test_df = pd.read_csv(IRIS_TEST, names=CSV_COLUMN_NAMES, header=0)
  test_features, test_label = test_df, test_df.pop(label_name)


 

  # Build SVM
  print("Building SVM using a {0} Kernel\n" .format(kernel))

  classifier = svm.SVC(decision_function_shape='ovo',kernel=kernel,  probability=True)
  

  
  # Train model.
  classifier.fit(train_features.values, train_label.values)

 

  # Evaluate accuracy.
  accuracy_score = classifier.score(train_features.values, train_label.values)

  print("\nTest Accuracy: {0:f}\n".format(accuracy_score))

  # Classify two new flower samples.
  # Classify two new flower samples.
  new_samples = np.array(
      [[6.4, 3.2, 4.5, 1.5],
       [5.8, 3.1, 5.0, 1.7],
       [6.4 ,2.8, 5.6 ,2.2],
       [5.0 , 2.3 ,3.3 ,1.0 ],
       [4.9 ,2.5 ,4.5 ,1.7 ],
       [4.9 ,3.1 ,1.5 ,0.1 ],
       [5.7,3.8 ,1.7 ,0.3]], dtype=np.float32)

 

  predicted_classes = classifier.predict(new_samples)
  probabilities = classifier.predict_log_proba(new_samples)
  predicted_class_names = []
  for i in range(len(predicted_classes)):
    predicted_class_names.append(iris_classes[int(predicted_classes[i])])
  
  #for i in range(len(predicted_classes)):
  #  predicted_classes[i] = int(predicted_classes[i])
  print(
      "New Samples, Class Predictions:    {}\n"
      .format(predicted_classes))
  print(
      "New Samples, Class Prediction Names:    {}\n"
      .format(predicted_class_names))
  result= {"training-size" :  train_features.shape,
           "test-size" : test_features.shape,
           "Model-type" : modelType,
           "model_dir"  : model_dir,
           "accuracy" : accuracy_score, 
           "prediction": predicted_classes,
           "prediction-names": predicted_class_names}

  return result  

In [ ]:
# Class SVM
# Other kernels = must be one of ‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, 
kernel = "rbf"   
iris_classifier_svm(kernel=kernel)

## Modify code above to take in a parameter for kernel='all'
## Print predictions from each SVM kernel


<li> You should get the following predictions:  [1, 2, 2, 1, 2, 0, 0]</li>
<li>You should get the follwing predictions: ['versicolor', 'virginica', 'virginica', 'versicolor', 'virginica', 'setosa', 'setosa']</li>
    </ul>


<img src="images/Logo.png" style="height:75px;width:120px;" align="left">
<h2> Combine SVM and DNN iris_classifiers into a single wrapper and print all predictions </h2>
<ul>
<li> Write a  function called iris_clasifier() that calls <em>iris_classifier_svm(kernel="all")<em> and <em>iris_classifier_dnn(..)</em></li>
<li> The function should return a JSON compaitable object (dictionary) that contains all the results from each classifier
<li> Print the Results object: Predictions from all SMV kernels, DecisionTree and DNN </li>
</ul>